In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import yfinance as yf
from pandas_datareader import data as pdr
import warnings

yf.pdr_override() 
plt.style.use("fivethirtyeight")
warnings.filterwarnings("ignore")

yesterday = datetime.now() - timedelta(1)
yesterday = datetime.strftime(yesterday, '%Y-%m-%d')
today = datetime.strftime(datetime.now(), '%Y-%m-%d')

In [2]:
## FUNCTIONS
def getMonthlyData(ticker="SPY", start="1994-01-01", end="2022-10-01"):
    data = yf.download(ticker, start, end)
    return data.resample('M').ohlc()['Close'].reset_index()
    
def func_13612WMomentum(data):
    data['shift1'] = data['close'].shift(1)
    data['shift3'] = data['close'].shift(3)
    data['shift6'] = data['close'].shift(6)
    data['shift12'] = data['close'].shift(12)
    data[['Date', 'close','shift1','shift3','shift6','shift12']]
    data = data.dropna(axis=0)
    data["13612W_momentum"] = (12*(data['close']/data['shift1']-1)) + (4*(data['close']/data['shift3']-1)) + (2*(data['close']/data['shift6']-1)) + (1*(data['close']/data['shift12']-1))
    return data

[Paper by Wouter J. Keller](https://papers.ssrn.com/sol3/papers.cfm?abstract_id=4166845)  
[Allocate Smartly](https://allocatesmartly.com/bold-asset-allocation/)

The strategy trades monthly. At the close on the last trading day of the month…

1. Start with the canary universe: S&P 500 (represented by SPY), developed intl equities (EFA), emerging market equities (EEM) and US aggregate bonds (AGG).
Calculate the “13612W” momentum of each asset. This is a multi-timeframe measure of momentum, calculated as follows:
(12 * (p0 / p1 – 1)) + (4 * (p0 / p3 – 1)) + (2 * (p0 / p6 – 1)) + (p0 / p12 – 1)
Where p0 = the price at today’s close, p1 = the price at the close of the previous month, etc.
Note how this approach overweights more recent months. Doing the math, the most recent 1-month change (p0/p1 – 1) determines 40% of the momentum score, while the most distant month (p11/p12 – 1) determines just ~2%.
2. If all canary assets have positive momentum, select from the offensive universe, otherwise select from the defensive universe.
Note how cautious this criterion is. If even one canary asset is exhibiting negative momentum (which has happened about 60% of the time), we shift to the defensive universe. The inclusion of agg. bonds may seem odd, but we’ve covered bonds as a positive predictor of risk asset returns a number of times, including here and here.
3. Select from within the appropriate universe (offensive or defensive) based on a slower relative momentum measurement. Calculate relative momentum as follows:  
`p0 / average(p0…p12)`  
I.e. today’s price divided by the average of the most recent 13 month-end prices.
4. If selecting from the offensive universe, select the 6 assets (balanced version) or 1 asset (aggressive version) with the highest relative momentum.
Warning: The aggressive version of the strategy is going all in on a single risk asset. That’s a dangerous approach. We suggest minimizing that risk by combining multiple unrelated strategies together in a combined portfolio (something our platform was built to tackle).
5. If selecting from the defensive universe, select the 3 assets with the highest relative momentum. If the relative momentum of the asset is less than that of US T-Bills (represented by ETF: BIL), instead place that portion of the portfolio in cash.
Here the strategy is using both relative and absolute momentum (aka “dual momentum”). More on this in a moment.
6. Equally-weight to all assets selected at the close. Rebalance the portfolio even if there isn’t a change in signal. Hold all positions until the end of the following month.

Asset universes:

- Canary – Aggressive and Balanced: S&P 500 (represented by SPY), emerging market equities (EEM), developed Intl equities (EFA) and US aggregate bonds (AGG)
- Offensive – Aggressive: Nasdaq 100 (QQQ), emerging market equities (EEM), developed intl equities (EFA) and US aggregate bonds (AGG)
- Offensive – Balanced: S&P 500 (SPY), Nasdaq 100 (QQQ), US small caps (IWM), Europe equities (VGK), Japan equities (EWJ), emerging market equities (EEM), US real estate (VNQ), commodities (DBC), gold (GLD), long-term US Treasuries (TLT), US high-yield bonds (HYG) and US corporate bonds (LQD)
- Defensive – Aggressive and Balanced: TIPS (TIP), commodities (DBC), US Treasury bills (BIL), intermediate-term US Treasuries (IEF), long-term US Treasuries (TLT), US corporate bonds (LQD) and US aggregate bonds (AGG)

In [3]:
## Canary Universe
# agg_data = getMonthlyData("AGG", end=today)
# spy_data = getMonthlyData("SPY", end=today)
# eem_data = getMonthlyData("EEM", end=today)
# efa_data = getMonthlyData("EFA", end=today)

canary = ["agg","spy","eem","efa"]
namespace = globals()
for i in canary:
    namespace['%s_data' % str(i)] = getMonthlyData(i, end=yesterday)

## Creating shift columns and calculating 13612W_momentum
agg_data = func_13612WMomentum(agg_data)
spy_data = func_13612WMomentum(spy_data)
eem_data = func_13612WMomentum(eem_data)
efa_data = func_13612WMomentum(efa_data)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [4]:
## Momentum
# If all canary assets have positive momentum, select from the offensive universe, otherwise select from the defensive universe.
Momentum = [agg_data['13612W_momentum'].iloc[-1], spy_data['13612W_momentum'].iloc[-1], eem_data['13612W_momentum'].iloc[-1], efa_data['13612W_momentum'].iloc[-1]]
negative_momentum = []
for i in Momentum:
    if i <= 0:
        negative_momentum.append(1)
    else:
        negative_momentum.append(0)
        
sum(negative_momentum)

4

In [5]:
if sum(negative_momentum) > 0:
    # DEFENSIVE
    pass
else:
    # OFFENSIVE
    pass

In [6]:
def relativeMomentum(data, col):
    for i in range(1,13):
        data['shift' + str(i)] = data[col].shift(i)
    data['relative_momentum'] = data['close'] / data.iloc[:, 1:len(data.columns)].mean(axis=1)
        
        
# test_data = eem_data.copy() 
# test_data = test_data[['Date', 'close']]
# relativeMomentum(test_data, 'close')
# test_data['relative_momentum'].tail(1)

In [15]:
offensive = ['spy','qqq','iwm','vgk','ewj','eem','vnq','bdc','gld','tlt','hyg','lqd']

namespace = globals()
for i in offensive:
    namespace['%s' % str(i)] = getMonthlyData(i, end=today)

for i in offensive:
    relativeMomentum(namespace['%s' % str(i)], 'close')    

ticker = []
rets = []
for i in offensive:
    ticker.append(str(i))
    rets.append(namespace['%s' % str(i)]['relative_momentum'].iloc[-1])

off_max = {'Ticker':ticker, 'Returns':rets}
off_max = pd.DataFrame(data=off_max)
off_max.sort_values(by='Returns', inplace=True, ascending=False)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [16]:
off_max.head(6)

,Ticker,Returns
7,bdc,1.055676
8,gld,0.949772
10,hyg,0.919041
2,iwm,0.888502
11,lqd,0.886139
0,spy,0.884228


In [ ]:
defensive = ['tip','dbc','bil','tlt','lqd','agg']
namespace = globals()
for i in canary:
    namespace['%s_data' % str(i)] = getMonthlyData(i, end=today)